# 1ª Lista de Exercícios

In [12]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import warnings
warnings.filterwarnings('ignore')


# Funções auxiliares

def describe_problem(problem):
    for variable in problem.variables():
        print(variable.name, "=", round(variable.varValue, 2))
    print(problem.objective.name, "=", round(problem.objective.value(), 2))


def Simplex(T, rotulos=[], base=[]):
    """
    Função para calcular o Tableau Simplex apresentando a sua evolução ao longo das iterações.

    Argumentos de entrada:
        T: numpy array representando o Tableau inicial

    Argumentos de Saída
        T: a tabela na última iteração do algoritmo

    Programador: Prof. Dr. Rooney Coelho
    """

    print('Tableu Simplex (inicial):')
    if rotulos == [] and base == []:
        print(T)
    else:
        print( pd.DataFrame(T, columns=rotulos, index=base ) )

    menor = -1
    it = 0
    while menor < 0:
        # Inicialização dos parâmetros (sobreescrever)
        pivo_linha = -1
        pivo_coluna = -1
        pivo = 0
        menor = T[0,:-1].min()

        if menor >=0:
            print('\nNenhum dos coeficientes da linha z associados com as variáveis não básicas é negativo assim essa tabela é ótima!')
            break
        else:
            it += 1
            print(f'\nIteração: {it}')
            # pega o menor elemento da primeira linha (função objetivo)
            pivo_coluna = T[0,:-1].argmin()

            aux = np.zeros(len(T)-1)
            i = 0
            for a,b in zip(T[1:,-1],T[1:,pivo_coluna]):
                aux[i] = a/b
                i+=1

            val = aux[aux>0].min()
            pivo_linha = np.argwhere(aux==val).item() + 1 # Soma um para a mesma referencia incluindo objetivo
            pivo = T[pivo_linha, pivo_coluna]
            print(f'Linha do pivô: {pivo_linha}, Coluna do pivô: {pivo_coluna}, Elemento pivô: {pivo}\n')

            # Nova linha do pivô = linha do pivô atual / elemento do pivô
            T[pivo_linha] = T[pivo_linha]/pivo

            # Todas as outras linhas, incluindo z
            # Nova linha=(Linha atual)−(seu coeficiente de coluna do pivô)×(Nova linha do pivô)

            nova_linha_pivo = T[pivo_linha]

            for i in range(len(T)):
                if i != pivo_linha:
                    T[i] -= T[i,pivo_coluna]*nova_linha_pivo

            print('Tableu Simplex:')
            if rotulos == [] and base == []:
                print(T)
            else:
                base[pivo_linha] = rotulos[pivo_coluna]
                print( pd.DataFrame(T, columns=rotulos, index=base ) )
    return T

## Execicio 1

Maximizar
<br>
$Z = 3x_1 + 6x_2$

Restrições

$9x_1 + 8x_2 \leq 72$
<br>$x_2 \leq 6$
<br>$-5x_1 + 4x_2 \leq 20$
<br>$2x_1 - 4x_2 \leq 20$
<br>$x_1, x_2 \geq 0$

Implementando em python usando pulp


In [13]:
from pulp import LpVariable, LpProblem, LpMaximize

problem1 = LpProblem("Execicio1", LpMaximize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')

# Função objetivo
problem1 += 3 * x1 + 6 * x2
# Restrições
problem1 += 9 * x1 + 8 * x2 <= 72
problem1 += x2 <= 6
problem1 += -5 * x1 + 4 * x2 <= 20
problem1 += 2 * x1 - 4 * x2 <= 20

problem1.solve()

describe_problem(problem1)

x1 = 2.67
x2 = 6.0
OBJ = 44.0


## Execicio 2

Maximizar
<br>
$Z = 10x_1 + 2x_2$ <br>

Restricoes

$10x_1 + 4x_2 \leq 40$
<br>$8x_1 + 2x_2 \geq 0$ <br>
$x_2 \leq 6$ <br>
$x_1 - 3x_2 \leq 0$ <br>
$x_1, x_2 \geq 0$

Solução

In [14]:
problem2 = LpProblem("Execicio2", LpMaximize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')

# Função objetivo
problem2 += 10 * x1 + 2 * x2

# Restrições
problem2 += 10 * x1 + 4 * x2 <= 40
problem2 += 8 * x1 + 2 * x2 >= 0
problem2 += x2 <= 6
problem2 += x1 - 3 * x2 <= 0

problem2.solve()

describe_problem(problem2)

x1 = 3.53
x2 = 1.18
OBJ = 37.65


## Execicio 3

Uma pequena manufatura produz dois modelos, Standard e Luxo, de um certo produto.
Cada unidade do modelo Standard requer 2 horas de lixação e 1 hora de polimento.
Cada unidade do modelo Luxo exige 2 horas de lixação e 3 horas de polimento.
A fábrica dispõe de 2 lixadoras e 3 polidoras, cada qual trabalhando 40 horas semanais.
As margens de lucro são $24 e $32, respectivamente, para cada unidade Standard e Luxo.
Não existem restrições de demanda para nenhum dos modelos.
Elabore um modelo de programação linear que nos permita calcular a produção semanal que maximize a margem total de lucro do fabricante.


Definindo a função objetivo e as restrições:
<br>
maximixar
$Z = 24x_1 + 32x_2$

restrições
$2x_1 + 2x_2 \leq 80$
$x_1 + 3x_2 \leq 120$
$x_1, x_2 \geq 0$

Implementando em python

In [15]:
problem3 = LpProblem("Execicio3", LpMaximize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')

# Função Objetivo
problem3 += 24 * x1 + 32 * x2

# Restrições
problem3 += 2 * x1 + 2 * x2 <= 80
problem3 += x1 + 3 * x2 <= 120

problem3.solve()

describe_problem(problem3)

x1 = 0.0
x2 = 40.0
OBJ = 1280.0


## Execicio 4

Um fazendeiro dispõe de 400ha cultiváveis com milho, trigo ou soja.
Cada hectare de milho exige 200 reais para preparação do terreno e 10 homens-dias de trabalho, e gera um lucro de 600 reais.
Um hectare de trigo implica custos de 240 reais para preparação do terreno e 16 homens-dias de trabalho, e dá um lucro de 700 reais.
Analogamente, um hectare de soja exige 140 reais e 12 homens-dias, e dá um lucro de 550 reais.
O fazendeiro dispõe de 80.000 reais para cobrir os custos de trabalho e 6.000 homens-dias de mão-de-obra.
Elabore um modelo de programação linear de modo a calcular a alocação de terra para os vários tipos de cultura com o objetivo de maximizar o lucro total.

maximizar
$Z = 600x_1 + 700x_2 + 550x_3$

restrições
$200x_1 + 240x_2 + 140x_3 \leq 80000$
$10x_1 + 16x_2 + 12x_3 \leq 6000$
$x_1 + x_2 + x_3 \leq 400$
$x_1, x_2, x_3 \geq 0$


In [16]:
problem4 = LpProblem("Execicio4", LpMaximize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')
x3 = LpVariable("x3", lowBound=0, cat='Continuous')

# Função objetivo
problem4 += 600 * x1 + 700 * x2 + 550 * x3

# Restrições
problem4 += 200 * x1 + 240 * x2 + 140 * x3 <= 80000
problem4 += 10 * x1 + 16 * x2 + 12 * x3 <= 6000
problem4 += x1 + x2 + x3 <= 400

problem4.solve()

describe_problem(problem4)

x1 = 0.0
x2 = 240.0
x3 = 160.0
OBJ = 256000.0


## Execicio 5

Encontre $x_1, x_2, x_3$ pelo método Simplex.

Maximizar
$Z = 8x_1 + 10x_2 + 6x_3$
Sujeito a:
$x_1 + x_3 \leq 400$
$4x_1 + 4x_2 + 2x_3 \leq 1200$
$3x_1 + 3x_2 \leq 600$
$x_1, x_2, x_3 \geq 0$

Adapatando para gerar igualdades:
$Z - 8x_1 - 10x_2 - 6x_3 = 0$
$x_1 + x_3 + f_1 = 400$
$4x_1 + 4x_2 + 2x_3 + f_2 = 1200$
$3x_1 + 3x_2 + f_3 = 600$







In [17]:
T = np.array([
    [1,-8,-10,-6,0,0,0,0],
    [0,1,0,1,1,0,0,400],
    [0,4,4,2,0,1,0,1200],
    [0,3,3,0,0,0,1,600]
], dtype=float)

rotulos = ['z', 'x1', 'x2', 'x3', 'f1', 'f2', 'f3', 'LD' ]
base = ['z', 'f1', 'f2', 'f3']

Simplex(T, rotulos, base);

Tableu Simplex (inicial):
      z   x1    x2   x3   f1   f2   f3      LD
z   1.0 -8.0 -10.0 -6.0  0.0  0.0  0.0     0.0
f1  0.0  1.0   0.0  1.0  1.0  0.0  0.0   400.0
f2  0.0  4.0   4.0  2.0  0.0  1.0  0.0  1200.0
f3  0.0  3.0   3.0  0.0  0.0  0.0  1.0   600.0

Iteração: 1
Linha do pivô: 3, Coluna do pivô: 2, Elemento pivô: 3.0

Tableu Simplex:
      z   x1   x2   x3   f1   f2        f3      LD
z   1.0  2.0  0.0 -6.0  0.0  0.0  3.333333  2000.0
f1  0.0  1.0  0.0  1.0  1.0  0.0  0.000000   400.0
f2  0.0  0.0  0.0  2.0  0.0  1.0 -1.333333   400.0
x2  0.0  1.0  1.0  0.0  0.0  0.0  0.333333   200.0

Iteração: 2
Linha do pivô: 2, Coluna do pivô: 3, Elemento pivô: 2.0

Tableu Simplex:
      z   x1   x2   x3   f1   f2        f3      LD
z   1.0  2.0  0.0  0.0  0.0  3.0 -0.666667  3200.0
f1  0.0  1.0  0.0  0.0  1.0 -0.5  0.666667   200.0
x3  0.0  0.0  0.0  1.0  0.0  0.5 -0.666667   200.0
x2  0.0  1.0  1.0  0.0  0.0  0.0  0.333333   200.0

Iteração: 3
Linha do pivô: 1, Coluna do pivô: 6, Element

## Execicio 6

Resolva o seguinte problema pelo método das duas fases

Minimizar
$Z = 4x_1 + 2x_2$
Sujeito a:
$4x_1 + 3x_2 \geq 6$
$6x_1 + 2x_2 = 6$
$2x_1 + 4x_2 \leq 6$
$x_1, x_2 \geq 0$

In [18]:
# TODO resolver pelo metodo das duas fases http://www.phpsimplex.com/pt/teoria_metodo_simplex.htm#:~:text=de%20equa%C3%A7%C3%B5es%20lineares).-,M%C3%A9todo%20das%20Duas%20Fases,(para%20evitar%20inconsist%C3%AAncias%20matem%C3%A1ticas).
from pulp import LpMinimize

problem6 = LpProblem("Execicio6", LpMinimize)

x1 = LpVariable("x1", lowBound=0, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, cat='Continuous')

# Função objetivo
problem6 += 4 * x1 + 2 * x2

# Restrições
problem6 += 4 * x1 + 3 * x2 >= 6
problem6 += 6 * x1 + 2 * x2 == 6
problem6 += 2 * x1 + 4 * x2 <= 6

problem6.solve()

describe_problem(problem6)


x1 = 0.6
x2 = 1.2
OBJ = 4.8


## Execicio 7

Encontre o problema Dual

Minimizar
$Z = 10x_1 + 6x_2 - 2y_3$
Sujeito a:
$2x_1 + 2x_2 + 2x_3 = 10$
$-2x_1 - x_2 - 3y_3 \geq −10$
$2x_1 - 4x_2 + 2x_3 \geq 4$
$x_1, x_2, x_3 \geq 0$

Dual
Maximizar
$D = 10y_1 - 10y_2 + 4y_3$
Sujeito a:
$2y_1 - 2y_2 + 2y_3 \leq 10$
$2y_1 -y_2 - 4y_3 \leq 6$
$2y_1 - 3y_2 + 2y_3 \leq −2$
$y_2, y_3 \geq 0$